## Descripción del Proyecto
El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

### Descripción de los datos

- `car_data.csv`: Descarga el conjunto de dato

Características

- `DateCrawled`: fecha en la que se descargó el perfil de la base de datos
- `VehicleType`: tipo de carrocería del vehículo
- `RegistrationYear` : año de matriculación del vehículo
- `Gearbox` : tipo de caja de cambios
- `Power` : potencia (CV)
- `Model` : modelo del vehículo
- `Mileage` : kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos)
- `RegistrationMonth` : mes de matriculación del vehículo
- `FuelType` :  tipo de combustible
- `Brand` : marca del vehículo
- `NotRepaired` : vehículo con o sin reparación
- `DateCreated` : fecha de creación del perfil
- `NumberOfPictures` : número de fotos del vehículo
- `PostalCode` : código postal del propietario del perfil (usuario)
- `LastSeen` : fecha de la última vez que el usuario estuvo activo

Objetivo

- `Price` : precio (en euros)

## Preparación de datos

### Inicialización Cargar librerias

In [1]:
import pandas as pd
import sklearn
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

import catboost as cb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc

In [2]:
vehicles = pd.read_csv('/datasets/car_data.csv')

Renombramos las columnas para que el código se vea más coherente con su estilo.

In [3]:
vehicles = vehicles.rename(columns={
    'DateCrawled': 'date_crawled', 
    'Price': 'price', 
    'VehicleType': 'vehicle_type', 
    'RegistrationYear': 'registration_year', 
    'Gearbox':'gearbox',
    'Power': 'power',
    'Model': 'model', 
    'Mileage':'mileage', 
    'RegistrationMonth':'registration_month', 
    'FuelType':'fuel_type', 
    'Brand':'brand',
    'NotRepaired':'not_repaired', 
    'DateCreated':'date_created', 
    'NumberOfPictures':'number_of_pictures', 
    'PostalCode':'postal_code',
    'LastSeen':'last_seen'
})

# actualizar indice
vehicles.reset_index(drop=True, inplace=True)

In [4]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   mileage             354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [5]:
vehicles.head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [6]:
# Definir el target 
target = "price"

# Eliminar columnas de fecha y codigo postal, etc.
drop_columns = [
    "date_crawled",
    "registration_month",
    "date_created",
    "postal_code",
    "last_seen",
    "number_of_pictures"
]
# Eliminar las columnas del dataframe
vehicles = vehicles.drop(columns=drop_columns)

# caracteristicas
features = [c for c in vehicles.columns if c not in drop_columns + [target]]


In [7]:
# Verificamos que las columnas se eliminaron 
vehicles.columns

Index(['price', 'vehicle_type', 'registration_year', 'gearbox', 'power',
       'model', 'mileage', 'fuel_type', 'brand', 'not_repaired'],
      dtype='object')

In [8]:
# Verificamos los tipos de datos, solo en las columnas de los features
vehicles[features].dtypes

vehicle_type         object
registration_year     int64
gearbox              object
power                 int64
model                object
mileage               int64
fuel_type            object
brand                object
not_repaired         object
dtype: object

In [9]:
# caracteristicas categoricas 
categorical_features = []

for col in vehicles[features].columns:
    if vehicles[col].dtype == "object":
        categorical_features.append(col)
        
numerical_features = [f for f in features if f not in categorical_features]

In [10]:
numerical_features

['registration_year', 'power', 'mileage']

In [11]:
categorical_features

['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'not_repaired']

In [12]:
# Verifica si hay valores nulos en los features
vehicles[features].isnull().sum()

vehicle_type         37490
registration_year        0
gearbox              19833
power                    0
model                19705
mileage                  0
fuel_type            32895
brand                    0
not_repaired         71154
dtype: int64

In [13]:
# Fila original
# vehicles[categorical_features].fillna('unknown') # reemplazar el dataframe original

# Fila modificada
vehicles[categorical_features] = vehicles[categorical_features].fillna('unknown') # reemplazar el dataframe original
vehicles

,price,vehicle_type,registration_year,gearbox,power,model,mileage,fuel_type,brand,not_repaired
0,480,unknown,1993,manual,0,golf,150000,petrol,volkswagen,unknown
1,18300,coupe,2011,manual,190,unknown,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...
354364,0,unknown,2005,manual,0,colt,150000,petrol,mitsubishi,yes
354365,2200,unknown,2005,unknown,0,unknown,20000,unknown,sonstige_autos,unknown
354366,1199,convertible,2000,auto,101,fortwo,125000,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,gasoline,volkswagen,no


In [14]:
# Identificar los valores 0 en la columna "power"
invalid_power_values = vehicles[vehicles['power'] == 0]

# Filtrar el DataFrame para excluir esos valores
# vehicles_cleaned = vehicles[vehicles['power'] != 0]
vehicles = vehicles[vehicles['power'] != 0]

## Entrenamiento del modelo 

In [15]:
# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
# Separa utilizando train_test_split.

train_df, test_df = train_test_split(
    vehicles, # vehicles_cleaned
    test_size = 0.2,
    random_state = 88
)

In [16]:
# Verificar la versión de la libreria
sklearn.__version__

'0.24.1'

In [17]:
# vehicles_copy = vehicles.copy()  # Crear una copia independiente
# vehicles_copy[categorical_features] = vehicles_copy[categorical_features].fillna('unknown')

In [18]:
# creamos una instancia del imputador.
# El argumento strategy='median' especifica que queremos reemplazar los valores faltantes de las característica numérica.
# imputer = SimpleImputer(strategy='median')

#  ajusta el imputador a las características numéricas del conjunto de entrenamiento (train_df)
# imputer.fit(train_df[numerical_features])

# aplica la imputación a las características numéricas del conjunto de entrenamiento.
# Los valores faltantes se reemplazan por la mediana correspondiente.
# train_df_nonulls = imputer.transform(train_df[numerical_features])
# test_df_nonulls = imputer.transform(test_df[numerical_features])

#  convierten las matrices resultantes en DataFrames de pandas.
# Luego, se asignan nombres de columnas a las características imputadas utilizando numerical_features
# train_df_nonulls = pd.DataFrame(train_df_nonulls)
# train_df_nonulls.columns = numerical_features

# test_df_nonulls = pd.DataFrame(test_df_nonulls)
# test_df_nonulls.columns = numerical_features

In [19]:
# vehicles[categorical_features].isnull().sum()

In [20]:
# Llenar los valores faltantes con la etiqueta "unknown"
# vehicles[categorical_features] = vehicles[categorical_features].fillna('unknown')

In [21]:
# vehicles[categorical_features].isnull().sum()

In [22]:
vehicles[categorical_features].shape

(314144, 6)

In [23]:
# train_df, test_df = train_test_split(
#     vehicles, # vehicles_cleaned
#     test_size = 0.2,
#     random_state = 88
# )

In [24]:
# Crear un objeto OrdinalEncoder
encoder = OrdinalEncoder()

# Ajustar el codificador a las características categóricas del conjunto de datos completo (vehicles)
# encoder.fit(vehicles[categorical_features])

# Aplicar el ordinal encoding a las características categóricas del conjunto de entrenamiento y prueba

# train_df_encoded = encoder.transform(vehicles[categorical_features])
# test_df_encoded = encoder.transform(vehicles[categorical_features])
encoder.fit(train_df[categorical_features])

train_df_encoded = train_df.copy()
train_df_encoded[categorical_features] = encoder.transform(train_df_encoded[categorical_features])

encoder.fit(test_df[categorical_features])

test_df_encoded = test_df.copy()
test_df_encoded[categorical_features] = encoder.transform(test_df_encoded[categorical_features])

# Convertir las matrices resultantes en DataFrames de pandas
# train_df_encoded = pd.DataFrame(train_df_encoded, columns=categorical_features)
# test_df_encoded = pd.DataFrame(test_df_encoded, columns=categorical_features)

In [25]:
train_df_encoded.head()

,price,vehicle_type,registration_year,gearbox,power,model,mileage,fuel_type,brand,not_repaired
343747,15000,4.0,2013,1.0,131,42.0,50000,2.0,24.0,1.0
253248,16200,2.0,2007,1.0,265,248.0,150000,6.0,2.0,0.0
285181,1500,8.0,2003,1.0,131,154.0,150000,2.0,10.0,0.0
139426,7300,0.0,2007,1.0,170,221.0,150000,2.0,38.0,0.0
165930,1690,4.0,2000,1.0,105,116.0,150000,6.0,38.0,0.0


In [26]:
# Verificamos valores nulos
train_df_encoded.isna().sum().sum()

0

In [27]:
# Llenar los valores faltantes con la etiqueta "unknown"
train_df_encoded = train_df_encoded.fillna(-100) 

# Verificamos valores nulos
train_df_encoded.isna().sum().sum()

0

In [28]:
# Verificamos valores nulos
test_df_encoded.isna().sum().sum()

0

In [29]:
# Llenar los valores faltantes con la etiqueta "unknown"
test_df_encoded = test_df_encoded.fillna(-100)

# Verificamos valores nulos
test_df_encoded.isna().sum().sum()

0

In [30]:
# train_df_encoded = pd.concat([vehicles.drop(columns=categorical_features), train_df_encoded], axis=1)
# train_df_encoded.head()

In [31]:
# test_df_encoded = pd.concat([vehicles.drop(columns=categorical_features), test_df_encoded], axis=1)
# test_df_encoded.head()

In [32]:
# # Verificamos cantidad de columnas
# train_df_encoded.shape

In [33]:
# # Verificamos cantidad de columnas
# test_df_encoded.shape

# OHE encoding

In [34]:
# Definir las características categóricas excluyendo 'model'
categorical_features = [col for col in vehicles.columns if vehicles[col].dtype == 'object' and col != 'model']

# tus características (matriz de datos) sin la columna 'model'
cat_features_x = vehicles.drop(columns=categorical_features)

# Aplicar OHE de forma más directa
train_df_ohe = pd.get_dummies(train_df, columns=categorical_features)
test_df_ohe = pd.get_dummies(test_df, columns=categorical_features)

# Eliminar la variable model de la tabla debido a que no está codificada y no es factible aplicarle OHE
test_df_ohe = test_df_ohe.drop(columns = 'model')
train_df_ohe = train_df_ohe.drop(columns = 'model')

In [35]:
# # One-Hot Encoding
# encoder = OneHotEncoder(drop='first', sparse=False)

# # Ajusta el codificador a las características categóricas del conjunto de entrenamiento
# encoder.fit(vehicles[categorical_features])

# # Transformar las características categóricas
# train_df_ohe = encoder.transform(vehicles[categorical_features].fillna('unknown'))
# test_df_ohe = encoder.transform(vehicles[categorical_features].fillna('unknown'))

# # Obtener los nombres de las nuevas columnas
# ohe_columns = encoder.get_feature_names(categorical_features)

# # Convertir las matrices resultantes en DataFrames de pandas
# train_df_ohe = pd.DataFrame(train_df_ohe, columns=ohe_columns)
# test_df_ohe = pd.DataFrame(test_df_ohe, columns=ohe_columns)


In [36]:
train_df_ohe.shape

(251315, 67)

In [37]:
test_df_ohe.shape

(62829, 67)

In [38]:
# # Verificamos los datos codificados

# # vehicles' es tu DataFrame original
# train_df, test_df = train_test_split(vehicles, test_size=0.2, random_state=88)

# # Aplicar la codificación One-Hot a las características categóricas
# encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# # Ajustar el codificador solo en el conjunto de entrenamiento
# train_encoded = encoder.fit_transform(train_df[categorical_features])
# test_encoded = encoder.transform(test_df[categorical_features])

# # Convertir a DataFrame y agregar de nuevo las columnas codificadas
# train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names(categorical_features))
# test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names(categorical_features))

# # Concatenar las columnas codificadas con los DataFrames originales
# train_df = pd.concat([train_df.reset_index(drop=True), train_encoded_df], axis=1)
# test_df = pd.concat([test_df.reset_index(drop=True), test_encoded_df], axis=1)

# # Eliminar las columnas originales categóricas si ya no son necesarias
# train_df.drop(['vehicle_type', 'gearbox', 'fuel_type', 'brand', 'not_repaired', 'model'], axis=1, inplace=True)
# test_df.drop(['vehicle_type', 'gearbox', 'fuel_type', 'brand', 'not_repaired', 'model'], axis=1, inplace=True)

In [39]:
print(train_df_encoded.shape)
print(test_df_encoded.shape)

(251315, 10)
(62829, 10)


In [40]:
print(train_df_ohe.shape)
print(test_df_ohe.shape)

(251315, 67)
(62829, 67)


## Análisis del modelo

### Modelo CatBoostRegressor

In [41]:
# Verificamos la versión y la documentación 
cb.__version__

'1.0.3'

In [42]:
# Entrenamos el modelo CatBoostRegressor
model_cb = cb.CatBoostRegressor(loss_function='RMSE', iterations=50)

In [43]:
%%time

# utilizamos train_df_encoded[features] y train_df_encoded[target] para entrenar modelo 
model_cb.fit(
    train_df_encoded[features].fillna(-100), # Rellenamos valores nulos con valor -100 fuera del rango para no afectarlo.
    train_df_encoded[target])

# utilizamos test_df  para la predicción 
preds_cb = model_cb.predict(test_df_encoded.fillna(-100))

# Calcular el error cuadratico medio, calcular cuadrada y redondeado a 4 decimales
print(f"""
RMSE_Catboost performance:
{round(mean_squared_error(test_df_encoded[target], preds_cb)**0.5, 4)}
""")

Learning rate set to 0.5
0:	learn: 3327.1717496	total: 104ms	remaining: 5.09s
1:	learn: 2739.1250486	total: 159ms	remaining: 3.81s
2:	learn: 2475.0432885	total: 211ms	remaining: 3.31s
3:	learn: 2321.2642328	total: 259ms	remaining: 2.98s
4:	learn: 2245.0367066	total: 306ms	remaining: 2.75s
5:	learn: 2192.5764065	total: 355ms	remaining: 2.6s
6:	learn: 2147.0547773	total: 404ms	remaining: 2.48s
7:	learn: 2122.6094267	total: 450ms	remaining: 2.36s
8:	learn: 2103.4015206	total: 497ms	remaining: 2.27s
9:	learn: 2062.3072762	total: 551ms	remaining: 2.2s
10:	learn: 2041.2662164	total: 598ms	remaining: 2.12s
11:	learn: 2024.0939709	total: 658ms	remaining: 2.08s
12:	learn: 2013.6973752	total: 704ms	remaining: 2s
13:	learn: 1999.7281602	total: 754ms	remaining: 1.94s
14:	learn: 1990.6031913	total: 805ms	remaining: 1.88s
15:	learn: 1982.4059704	total: 857ms	remaining: 1.82s
16:	learn: 1972.0698994	total: 903ms	remaining: 1.75s
17:	learn: 1963.3756313	total: 955ms	remaining: 1.7s
18:	learn: 1953.700

### Modelo LinearRegression

In [ ]:
train_df_ohe.drop(columns = 'price')

In [ ]:
# Ajustar el modelo
model_lr = LinearRegression()
model_lr.fit(train_df_ohe.drop(columns = 'price'), train_df_ohe['price'])  

In [ ]:
%%time

# Ajustar el modelo
model_lr = LinearRegression()
model_lr.fit(train_df_ohe.drop(columns = 'price'), train_df_ohe['price'])  

# Utilizar test_df para la predicción
preds_lr = model_lr.predict(test_df_ohe.drop(columns = 'price'))

# Calcular el error cuadrático medio, calcular raíz cuadrada y redondear a 4 decimales
rmse_lr = round(mean_squared_error(test_df_ohe['price'], preds_lr) ** 0.5, 4)
print(f"RMSE_LinearRegression:\n{rmse_lr}")

### Modelo RandomForestRegressor

In [ ]:
%%time

# Modelo de Random Forest
model_rf = RandomForestRegressor(random_state=88)
model_rf.fit(train_df_ohe.drop(columns = 'price'), train_df_ohe['price'])  

# utilizamos test_df para la predicción 
preds_rf = model_rf.predict(test_df_ohe.drop(columns = 'price'))

# Calcular el error cuadratico medio, calcular cuadrada y redondeado a 4 decimales
print(f"""
RMSE_RandomForestRegressor:
{round(mean_squared_error(test_df_ohe['price'], preds_rf)**0.5, 4)}
""")

## Conclusiones
- El mejor modelo de entrenamiento es Catboost performance para el analisis en tiempo, velocidad y exactitud.

###  RMSE_LinearRegression: 3352.4402

CPU times: user 2.31 s, sys: 446 ms, total: 2.76 s
Wall time: 2.71 s

### RMSE_Catboost performance: 1898.5014

CPU times: user 2.78 s, sys: 8.86 ms, total: 2.79 s
Wall time: 3.01 s

### RMSE_RandomForestRegressor: 3569.0812

CPU times: user 3min 10s, sys: 43.4 ms, total: 3min 10s
Wall time: 3min 11s


# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores
- [x]  Las celdas con el código han sido colocadas en orden de ejecución
- [x]  Los datos han sido descargados y preparados
- [x]  Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos